In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np
import plotly

In [2]:
# load business.json
filepath = '../../yelp_dataset'

business = []
for l in open(filepath+"/business.json", encoding="utf8").readlines():
    business.append(json.loads(l))
df_business = pd.DataFrame.from_records(business)


In [3]:
# load data from previous preprocessing/EDA
filepath = '../submission3/'

df_restaurant_tips = pd.read_json(filepath+"restaurant_tips.json", encoding="utf8")

In [4]:
# Filter businesses that are only in the 'Restaurant' category
def check_for_rest(row):
    category = row['categories']
    if category:
        tokens = category.split(', ')
        return 'Restaurants' in tokens
    return False

df_business['is_restaurant'] = df_business.apply(check_for_rest, axis=1)

food_businesses = df_business[df_business['is_restaurant'] == True]

In [5]:
# CUISINES WE WILL EXPLORE:
# Fast Food, American (Traditional), American (New)
def is_fast_food(row):
    category = row['categories']
    if category:
        tokens = category.split(', ')
        return 'Fast Food' in tokens
    return False

def is_american_traditional(row):
    category = row['categories']
    if category:
        tokens = category.split(', ')
        return 'American (Traditional)' in tokens
    return False

def is_american_new(row):
    category = row['categories']
    if category:
        tokens = category.split(', ')
        return 'American (New)' in tokens
    return False

# Fast Food
food_businesses['is_fast_food'] = food_businesses.apply(is_fast_food, axis=1)
df_fast_food = food_businesses[food_businesses['is_fast_food'] == True]

# American (Traditional)
food_businesses['is_american_t'] = food_businesses.apply(is_american_traditional, axis=1)
df_american_t = food_businesses[food_businesses['is_american_t'] == True]

# American (New)
food_businesses['is_american_n'] = food_businesses.apply(is_american_new, axis=1)
df_american_n = food_businesses[food_businesses['is_american_n'] == True]

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [6]:
# get tips for Fast Food mapped to business_id
fast_food_ids = set(df_fast_food.business_id.unique())
american_t_ids = set(df_american_t.business_id.unique())
american_n_ids = set(df_american_n.business_id.unique())

# group tips

cuisine_tips_fast_food = df_restaurant_tips[df_restaurant_tips['business_id'].isin(fast_food_ids)]
cuisine_tips_american_t = df_restaurant_tips[df_restaurant_tips['business_id'].isin(american_t_ids)]
cuisine_tips_american_n = df_restaurant_tips[df_restaurant_tips['business_id'].isin(american_n_ids)]


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def get_sentiment(sentence):
    dic = analyser.polarity_scores(sentence)
    # dic -> {'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'compound': -0.5423}
    # for some reason only allows dictionary return type
    return dic

def get_positive_sentiment(row):
    sentence = row['text']
    dic = get_sentiment(sentence)
    return dic['compound'] #score is normalized between 0-1

In [8]:
cuisine_tips_fast_food['text_sentiment'] = cuisine_tips_fast_food.apply(get_positive_sentiment, axis=1)
cuisine_tips_american_t['text_sentiment'] = cuisine_tips_american_t.apply(get_positive_sentiment, axis=1)
cuisine_tips_american_n['text_sentiment'] = cuisine_tips_american_n.apply(get_positive_sentiment, axis=1)

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/leannahue/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [9]:
# Meal times
breakfast_times = set([5,6,7,8,9,10])
lunch_times = set([11,12,13,14,15,16])
dinner_time = set([17,18,19,20,21,22])

In [10]:
# Breakfast tips
fast_food_tips_breakfast = cuisine_tips_fast_food[cuisine_tips_fast_food['local_hour'].isin(breakfast_times)]
american_t_tips_breakfast = cuisine_tips_american_t[cuisine_tips_american_t['local_hour'].isin(breakfast_times)]
american_n_tips_breakfast = cuisine_tips_american_n[cuisine_tips_american_n['local_hour'].isin(breakfast_times)]

In [11]:
# Lunch tips
fast_food_tips_lunch = cuisine_tips_fast_food[cuisine_tips_fast_food['local_hour'].isin(lunch_times)]
american_t_tips_lunch = cuisine_tips_american_t[cuisine_tips_american_t['local_hour'].isin(lunch_times)]
american_n_tips_lunch = cuisine_tips_american_n[cuisine_tips_american_n['local_hour'].isin(lunch_times)]

In [12]:
# Dinner tips
fast_food_tips_dinner = cuisine_tips_fast_food[cuisine_tips_fast_food['local_hour'].isin(dinner_time)]
american_t_tips_dinner = cuisine_tips_american_t[cuisine_tips_american_t['local_hour'].isin(dinner_time)]
american_n_tips_dinner = cuisine_tips_american_n[cuisine_tips_american_n['local_hour'].isin(dinner_time)]

In [13]:
def find_avg_tip_sentiment(row, df_tips):
    business_id = row['business_id']
    tips_with_this_business = df_tips[df_tips['business_id'] == business_id]
    return tips_with_this_business['text_sentiment'].mean()

In [14]:
# Breakfast sentiments
fast_food_businesses_breakfast = pd.DataFrame({'business_id': list(fast_food_tips_breakfast['business_id'].unique())})
fast_food_businesses_breakfast['avg_tip_sentiment'] = \
    fast_food_businesses_breakfast.apply(lambda row: find_avg_tip_sentiment(row, fast_food_tips_breakfast), axis=1)

american_t_businesses_breakfast = pd.DataFrame({'business_id': list(american_t_tips_breakfast['business_id'].unique())})
american_t_businesses_breakfast['avg_tip_sentiment'] = \
    american_t_businesses_breakfast.apply(lambda row: find_avg_tip_sentiment(row, american_t_tips_breakfast), axis=1)

american_n_businesses_breakfast = pd.DataFrame({'business_id': list(american_n_tips_breakfast['business_id'].unique())})
american_n_businesses_breakfast['avg_tip_sentiment'] = \
    american_n_businesses_breakfast.apply(lambda row: find_avg_tip_sentiment(row, american_n_tips_breakfast), axis=1)

In [15]:
# Lunch sentiments
fast_food_businesses_lunch = pd.DataFrame({'business_id': list(fast_food_tips_lunch['business_id'].unique())})
fast_food_businesses_lunch['avg_tip_sentiment'] = \
    fast_food_businesses_lunch.apply(lambda row: find_avg_tip_sentiment(row, fast_food_tips_lunch), axis=1)

american_t_businesses_lunch = pd.DataFrame({'business_id': list(american_t_tips_lunch['business_id'].unique())})
american_t_businesses_lunch['avg_tip_sentiment'] = \
    american_t_businesses_lunch.apply(lambda row: find_avg_tip_sentiment(row, american_t_tips_lunch), axis=1)

american_n_businesses_lunch = pd.DataFrame({'business_id': list(american_n_tips_lunch['business_id'].unique())})
american_n_businesses_lunch['avg_tip_sentiment'] = \
    american_n_businesses_lunch.apply(lambda row: find_avg_tip_sentiment(row, american_n_tips_lunch), axis=1)

In [16]:
# Dinner sentiments
fast_food_businesses_dinner = pd.DataFrame({'business_id': list(fast_food_tips_dinner['business_id'].unique())})
fast_food_businesses_dinner['avg_tip_sentiment'] = \
    fast_food_businesses_dinner.apply(lambda row: find_avg_tip_sentiment(row, fast_food_tips_dinner), axis=1)

american_t_businesses_dinner = pd.DataFrame({'business_id': list(american_t_tips_dinner['business_id'].unique())})
american_t_businesses_dinner['avg_tip_sentiment'] = \
    american_t_businesses_dinner.apply(lambda row: find_avg_tip_sentiment(row, american_t_tips_dinner), axis=1)

american_n_businesses_dinner = pd.DataFrame({'business_id': list(american_n_tips_dinner['business_id'].unique())})
american_n_businesses_dinner['avg_tip_sentiment'] = \
    american_n_businesses_dinner.apply(lambda row: find_avg_tip_sentiment(row, american_n_tips_dinner), axis=1)

In [23]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=list(fast_food_businesses_breakfast['avg_tip_sentiment']),
    histnorm='percent',
    name='Fast Food', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#8dd3c7'
))
fig.add_trace(go.Histogram(
    x=list(american_t_businesses_breakfast['avg_tip_sentiment']),
    histnorm='percent',
    name='American (Traditional)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#bebada'
))
fig.add_trace(go.Histogram(
    x=list(american_n_businesses_breakfast['avg_tip_sentiment']),
    histnorm='percent',
    name='American (New)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#fb8072'
))

fig.update_layout(
    title_text='Distribution of Average Sentiment for Businesses (Breakfast)', # title of plot
    xaxis_title_text='Average Tip Sentiment', # xaxis label
    yaxis_title_text='Percentage of Occurrences', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

plotly.offline.plot(fig)

'temp-plot.html'

In [24]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=list(fast_food_businesses_lunch['avg_tip_sentiment']),
    histnorm='percent',
    name='Fast Food', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#8dd3c7'
))
fig.add_trace(go.Histogram(
    x=list(american_t_businesses_lunch['avg_tip_sentiment']),
    histnorm='percent',
    name='American (Traditional)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#bebada'
))
fig.add_trace(go.Histogram(
    x=list(american_n_businesses_lunch['avg_tip_sentiment']),
    histnorm='percent',
    name='American (New)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#fb8072'
))

fig.update_layout(
    title_text='Distribution of Average Sentiment for Businesses (Lunch)', # title of plot
    xaxis_title_text='Average Tip Sentiment', # xaxis label
    yaxis_title_text='Percentage of Occurrences', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

plotly.offline.plot(fig)

'temp-plot.html'

In [25]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=list(fast_food_businesses_dinner['avg_tip_sentiment']),
    histnorm='percent',
    name='Fast Food', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#8dd3c7'
))
fig.add_trace(go.Histogram(
    x=list(american_t_businesses_dinner['avg_tip_sentiment']),
    histnorm='percent',
    name='American (Traditional)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#bebada'
))
fig.add_trace(go.Histogram(
    x=list(american_n_businesses_dinner['avg_tip_sentiment']),
    histnorm='percent',
    name='American (New)',
    xbins=dict(
        start=-1.0,
        end=1.0,
        size=0.1
    ),
    marker_color='#fb8072'
))

fig.update_layout(
    title_text='Distribution of Average Sentiment for Businesses (Dinner)', # title of plot
    xaxis_title_text='Average Tip Sentiment', # xaxis label
    yaxis_title_text='Percentage of Occurrences', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

plotly.offline.plot(fig)

'temp-plot.html'